In [1]:
import numpy as np
import nibabel as ni
import pandas as pd

In [22]:
vlsm = ni.load("vlsm_mgh.nii")
vlsm_data = np.array(vlsm.dataobj)

In [23]:
lnm = ni.load("mgh_palm/_vox_tstat.nii")
lnm_data = np.array(lnm.dataobj)

In [24]:
wm = ni.load("mgh_palm_dcn/_vox_tstat.nii")
wm_data = np.array(wm.dataobj)

In [5]:
total_len = len(vlsm_data)*len(vlsm_data[0])*len(vlsm_data[0, 0])
per10 = int(total_len*0.1)
per5 = int(total_len*0.05)
per1 = int(total_len*0.01)

In [13]:
affine = vlsm.affine

In [7]:
def change_voxels(zeros, sorted_list):
    for i in range(len(sorted_list)):
        x = sorted_list[i][1][0]
        y = sorted_list[i][1][1]
        z = sorted_list[i][1][2]
        zeros[x, y, z] = sorted_list[i][0]
    return zeros

In [22]:
def peak_regions_imgs(data, mode):
    voxels = []
    for x in range(len(data)):
        for y in range(len(data[x])):
            for z in range(len(data[x, y])):
                voxels.append((data[x, y, z], tuple([x, y, z])))
    sorted_voxels = sorted(voxels, reverse=True)
    per10_list = sorted_voxels[0:per10]
    per5_list = sorted_voxels[0:per5]
    per1_list = sorted_voxels[0:per1]
    arr = np.zeros(tuple([len(data), len(data[0]), len(data[0, 0])]))
    arr_per10 = change_voxels(arr, per10_list)
    arr_per5 = change_voxels(arr, per5_list)
    arr_per1 = change_voxels(arr, per1_list)
    img_per10 = ni.nifti1.Nifti1Image(arr_per10, affine=affine)
    img_per5 = ni.nifti1.Nifti1Image(arr_per5, affine=affine)
    img_per1 = ni.nifti1.Nifti1Image(arr_per1, affine=affine)
#     ni.save(img_per10, "peak_regions/"+mode+"_per10.nii")
#     ni.save(img_per5, "peak_regions/"+mode+"_per5.nii")
#     ni.save(img_per1, "peak_regions/"+mode+"_per1.nii")

In [23]:
peak_regions_imgs(vlsm_data, "vlsm")
peak_regions_imgs(lnm_data, "lnm")
peak_regions_imgs(wm_data, "wm")

In [28]:
def peak_regions_nonzeros(data, mode):
    voxels = []
    total_len = 0
    for x in range(len(data)):
        for y in range(len(data[x])):
            for z in range(len(data[x, y])):
                voxels.append((data[x, y, z], tuple([x, y, z])))
                if data[x, y, z] > 0:
                    total_len += 1
    sorted_voxels = sorted(voxels, reverse=True)
    per10 = int(total_len*0.1)
    per5 = int(total_len*0.05)
    per1 = int(total_len*0.01)
    per10_list = sorted_voxels[0:per10]
    per5_list = sorted_voxels[0:per5]
    per1_list = sorted_voxels[0:per1]
    arr = np.zeros(tuple([len(data), len(data[0]), len(data[0, 0])]))
    arr_per10 = change_voxels(arr, per10_list)
    arr_per5 = change_voxels(arr, per5_list)
    arr_per1 = change_voxels(arr, per1_list)
    img_per10 = ni.nifti1.Nifti1Image(arr_per10, affine=affine)
    img_per5 = ni.nifti1.Nifti1Image(arr_per5, affine=affine)
    img_per1 = ni.nifti1.Nifti1Image(arr_per1, affine=affine)
#     ni.save(img_per10, "peak_regions/"+mode+"_nz_per10.nii")
#     ni.save(img_per5, "peak_regions/"+mode+"_nz_per5.nii")
#     ni.save(img_per1, "peak_regions/"+mode+"_nz_per1.nii")

In [29]:
peak_regions_nonzeros(vlsm_data, "vlsm")
peak_regions_nonzeros(lnm_data, "lnm")
peak_regions_nonzeros(wm_data, "wm")

In [28]:
def peak_regions_n(data, mode, n):
    voxels = []
    for x in range(len(data)):
        for y in range(len(data[x])):
            for z in range(len(data[x, y])):
                voxels.append((data[x, y, z], tuple([x, y, z])))
    sorted_voxels = sorted(voxels, reverse=True)
    list_n = sorted_voxels[0:n]
    arr = np.zeros(tuple([len(data), len(data[0]), len(data[0, 0])]))
    arr_n = change_voxels(arr, list_n)
    img_n = ni.nifti1.Nifti1Image(arr_n, affine=affine)
    ni.save(img_n, "peak_regions/mgh_"+mode+"_"+str(n)+"_.nii")

In [29]:
peak_regions_n(vlsm_data, "vlsm", 10000)
peak_regions_n(lnm_data, "lnm", 10000)
peak_regions_n(wm_data, "wm", 10000)
peak_regions_n(vlsm_data, "vlsm", 5000)
peak_regions_n(lnm_data, "lnm", 5000)
peak_regions_n(wm_data, "wm", 5000)

In [25]:
def peak_overlap(n):
    vlsm_voxels = []
    lnm_voxels = []
    wm_voxels = []
    for x in range(len(vlsm_data)):
        for y in range(len(vlsm_data[x])):
            for z in range(len(vlsm_data[x, y])):
                vlsm_voxels.append(((vlsm_data[x, y, z]), tuple([x, y, z])))
                lnm_voxels.append(((lnm_data[x, y, z]), tuple([x, y, z])))
                wm_voxels.append(((wm_data[x, y, z]), tuple([x, y, z])))
    sorted_vlsm = sorted(vlsm_voxels, reverse=True)
    sorted_lnm = sorted(lnm_voxels, reverse=True)
    sorted_wm = sorted(wm_voxels, reverse=True)
    vlsm_list = sorted_vlsm[0:n]
    lnm_list = sorted_lnm[0:n]
    wm_list = sorted_wm[0:n]
    vlsm_set = set([sorted_vlsm[k][1] for k in range(n)])
    lnm_set = set([sorted_lnm[k][1] for k in range(n)])
    wm_set = set([sorted_wm[k][1] for k in range(n)])
    arr = np.zeros(tuple([len(vlsm_data), len(vlsm_data[0]), len(vlsm_data[0, 0])]))
    for x in range(len(vlsm_data)):
        for y in range(len(vlsm_data[x])):
            for z in range(len(vlsm_data[x, y])):
                if tuple([x, y, z]) in vlsm_set and tuple([x, y, z]) in lnm_set and tuple([x, y, z]) in wm_set:
                    arr[x, y, z] = 3
                elif (tuple([x, y, z]) in vlsm_set and tuple([x, y, z]) in lnm_set) or (tuple([x, y, z]) in vlsm_set and tuple([x, y, z]) in wm_set) or (tuple([x, y, z]) in wm_set and tuple([x, y, z]) in lnm_set):
                    arr[x, y, z] = 2
                elif tuple([x, y, z]) in vlsm_set or tuple([x, y, z]) in lnm_set or tuple([x, y, z]) in wm_set:
                    arr[x, y, z] = 1
    img = ni.nifti1.Nifti1Image(arr, affine=affine)
    ni.save(img, "peak_regions/mgh_overlap_"+str(n)+".nii")

In [26]:
peak_overlap(10000)

In [27]:
peak_overlap(5000)

In [9]:
peak_overlap(1000)